In [1]:
import asyncio
import pandas as pd
from app.tools.context_store import AnalysisContext 
from deepagents import create_deep_agent
from app.tools.metrics_agent_tools import get_dataset_health_check, query_anomalous_ids, run_ml_inference_pipeline
from app.tools.data_tools import *

orchestrator_tools = [
      query_mongo_requests, 
      query_sql_campaigns, 
      list_avaiable_datasets, 
      inspect_file_schema, 
      load_dataset_into_context, 
      check_context_status, 
      inspect_file_schema
]


async def test_full_workflow():
    print("\n🔹 --- INICIANDO TESTE DE INTEGRAÇÃO (SEM LLM) ---")

    # ==============================================================================
    # 1. SIMULAÇÃO DO ORQUESTRADOR (Discovery & Ingestion)
    # ==============================================================================
    print("\nTesting: 1. Orchestrator - Discovery (SQL)")
    
    # Simula o LLM chamando a tool com argumentos
    # Nota: Se suas tools usam @tool, use .invoke() ou chame a função decorada diretamente dependendo da versão do LangChain
    try:
        # Tenta listar campanhas do Google
        campaigns_str = await query_sql_campaigns.ainvoke({"traffic_source": "google", "limit": 2})
        print(f"✅ SQL Result: {campaigns_str}")
        
        # HACK PARA O TESTE:
        # Como não temos o LLM para ler a string e escolher o hash, vamos pegar um hash 'fake' 
        # ou extrair da string se o banco estiver conectado.
        # Para este teste, vou assumir que você pegou um hash válido do log acima.
        target_hash = "uw0qfu4a1r" 
        print(f"👉 Usando Hash Alvo: {target_hash}")

    except Exception as e:
        print(f"❌ SQL Failed: {e}")
        return

    print("\nTesting: 2. Orchestrator - Ingestion (Mongo -> Context)")
    try:
        # Simula o carregamento

        status_msg = await query_mongo_requests.ainvoke({
            "hash": target_hash,
            # "hashes": campaigns_str, 
            "traffic_source": "google"
        })
        print(f"✅ Load Result: {status_msg}")
        
        # VERIFICAÇÃO DE ESTADO (Crucial!)
        # Vamos espiar dentro do Singleton para ver se funcionou
        try:
            print("Status: ", AnalysisContext.get_status())
            # print(f"🔎 VERIFICAÇÃO: Contexto contém {len(df)} linhas. Colunas: {list(df.columns[:3])}...")
        except ValueError:
            print("❌ VERIFICAÇÃO FALHOU: Contexto está vazio!")
            return

    except Exception as e:
        print(f"❌ Mongo Load Failed: {e}")
        # SE VOCÊ NÃO TEM BANCO RODANDO AGORA, DESCOMENTE A LINHA ABAIXO PARA MOCKAR DADOS:
        # mock_data_loading() 
        return

    # ==============================================================================
    # 2. SIMULAÇÃO DO SUB-AGENTE (Analysis & Inference)
    # ==============================================================================
    print("\nTesting: 3. Sub-agent - ML Execution")
    try:
        # O agente chama sem argumentos, pois pega do Contexto
        inference_summary = run_ml_inference_pipeline.invoke({}) 
        print(f"✅ Inference Result: {inference_summary}")
    except Exception as e:
        print(f"❌ ML Pipeline Failed: {e}")
        return

    print("\nTesting: 4. Sub-agent - Health Check")
    try:
        health_stats = get_dataset_health_check.invoke({})
        print(f"✅ Health Stats: {health_stats}")
    except Exception as e:
        print(f"❌ Health Check Failed: {e}")

    print("\nTesting: 5. Sub-agent - Query Anomalies")
    try:
        # Testa buscar IDs com baixa confiança
        anomalies = query_anomalous_ids.invoke({"criteria": "low_trust", "threshold": 0.5})
        print(f"✅ Found {len(anomalies)} anomalies. Sample IDs: {anomalies[:5]}")
    except Exception as e:
        print(f"❌ Query Failed: {e}")

    print("\n🔹 --- TESTE FINALIZADO ---")

# --- MOCK OPCIONAL (Se você não tiver o Mongo rodando localmente) ---
def mock_data_loading():
    print("⚠️ MOCKING DATA LOADING...")
    data = {
        "id": range(100),
        "user_agent": ["Mozilla/5.0"] * 50 + ["Googlebot"] * 50,
        "url": ["/home"] * 100,
        "label": [1]*50 + [0]*50 # 1=Human, 0=Bot
    }
    df = pd.DataFrame(data)
    AnalysisContext.set_mongo_data(df, "google", "mock_file.json")
    print("✅ Mock data loaded into Context.")

if __name__ == "__main__":
    await test_full_workflow()

c:\Users\Camille\Documents\TWR\deep_agents_twr\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Camille\Documents\TWR\deep_agents_twr\.venv\Lib\site-packages\motor\core.py:171: UserWarning: You appear to be connected to a DocumentDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/documentdb
  delegate = self.__delegate_class__(*args, **kwargs)



🔹 --- INICIANDO TESTE DE INTEGRAÇÃO (SEM LLM) ---

Testing: 1. Orchestrator - Discovery (SQL)
✅ SQL Result: ['nod1rhsrf6', '61jp3j1p4f']
👉 Usando Hash Alvo: uw0qfu4a1r

Testing: 2. Orchestrator - Ingestion (Mongo -> Context)
DEBUG [Context]: Mongo Data Loaded. Rows: 1000
✅ Load Result: SUCCESS: Loaded 1000 requests into AnalysisContext.
Sources: google | Hashes: 1
Action Required: Delegate to 'Metrics Analyst' agent to run ML inference now.
Status:  Mongo Raw: 1000 rows (None) | ML Processed: Pending

Testing: 3. Sub-agent - ML Execution


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 422.95it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Batches: 100%|██████████| 16/16 [00:30<00:00,  1.91s/it]


{'id': tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63]), 'x': tensor([[-0.0727, -0.0736,  0.0159,  ...,  0.0729,  0.0176,  0.0459],
        [-0.0695, -0.0018, -0.0149,  ..., -0.0260, -0.0205,  0.0071],
        [-0.0709, -0.0706,  0.0224,  ...,  0.0794,  0.0144,  0.0460],
        ...,
        [-0.0617, -0.0776,  0.0268,  ...,  0.0753,  0.0175,  0.0449],
        [-0.0628, -0.0510, -0.0232,  ...,  0.0412, -0.0091,  0.0307],
        [-0.0711, -0.0332, -0.0414,  ...,  0.0217, -0.0258,  0.0343]]), 'label': tensor([0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
        0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0]), 'raw_text': ['UA: Mozilla\\/5.0 (Lin